# Fitting

There are two built in fitting engines, `lmfit` and `bumps`.

In [ ]:
# Import all the packages
import numpy as np
from easyDiffractionLib.sample import Sample
from easyDiffractionLib import Phases
from easyDiffractionLib.interface import InterfaceFactory as Calculator

from easyDiffractionLib.Elements.Experiments.Pattern import Pattern1D
from easyDiffractionLib.Profiles.P1D import Instrument1DCWParameters

from easyscience.fitting.fitter import Fitter

import matplotlib.pyplot as plt

## Preparing the sample and data

Load structure from a CIF file

In [ ]:
calculator = Calculator()
phase = Phases.from_cif_file('PbSO4.cif')
sample = Sample(phases=phase, parameters=Instrument1DCWParameters.default(), calculator=calculator)

Visualise the structure

In [ ]:
import py3Dmol
viewer = py3Dmol.view()
viewer.addModel(phase[0].to_cif_str(),'cif')
viewer.setStyle({'sphere':{'colorscheme':'Jmol','scale':.2},'stick':{'colorscheme':'Jmol', 'radius': 0.1}})
viewer.addUnitCell()
viewer.replicateUnitCell(2,2,1)
viewer.zoomTo()

Load experimental data from a file

In [ ]:
file_path = 'PbSO4_neutrons_short.xye'
data_x, data_y, data_e = np.loadtxt(file_path, unpack=True)

data_y = data_y/100.0

# Generate the simulation y-data
sim_y_data = calculator.fit_func(data_x)


In [ ]:
%matplotlib notebook
plt.plot(data_x, data_y, label='Experimental')
plt.plot(data_x, sim_y_data, label='Starting point')
plt.legend()

The charts do not match very well, because our simulation did not include any parameters related to the experiment.
Let's assign some decent values then.

In [ ]:
sample.parameters.wavelength = 1.912
sample.parameters.u_resolution = 1.4
sample.parameters.v_resolution = -0.42
sample.parameters.w_resolution = 0.38
sample.parameters.x_resolution = 0.0
sample.parameters.y_resolution = 0.0

sim_y_data = calculator.fit_func(data_x)

%matplotlib notebook
plt.plot(data_x, data_y, label='Experimental')
plt.plot(data_x, sim_y_data, label='Starting point')
plt.legend()

This looks much better now - experimental and theoretical peaks seem to be very close but there is no background included in our simulation.

In [ ]:
from easyDiffractionLib.Elements.Backgrounds.Point import PointBackground, BackgroundPoint

bg = PointBackground(linked_experiment='PbSO4')
bg.append(BackgroundPoint.from_pars(data_x[0], 2))
bg.append(BackgroundPoint.from_pars(data_x[-1], 2))

sample.set_background(bg)

In [ ]:
sim_y_data = calculator.fit_func(data_x)
%matplotlib notebook
plt.plot(data_x, data_y, label='Experimental')
plt.plot(data_x, sim_y_data, label='Starting point')
plt.legend()

These two charts look close enough to attempt fitting.

## Fitting to the data

Initalize the fitting engine and define parameters to optimize

In [ ]:
f = Fitter(sample, calculator.fit_func)

# Vary the scale and the BG points
sample.pattern.scale.fixed = False
sample.parameters.resolution_u.fixed = False
sample.parameters.resolution_v.fixed = False
sample.parameters.resolution_w.fixed = False
sample.backgrounds[0][0].y.fixed = False
sample.backgrounds[0][1].y.fixed = False

Perform the fit

In [ ]:
result = f.fit(data_x, data_y, weights=1/data_e)

if result.success:
    print("The fit has been successful: {}".format(result.success))
    print("The gooodness of fit is: {}".format(result.goodness_of_fit))
    
sim_y_data = calculator.fit_func(data_x)

In [ ]:
%matplotlib notebook
plt.plot(data_x, data_y, label='Experimental')
plt.plot(data_x, sim_y_data, label='Best Fit')
plt.legend()

## Fitted parameters

In [ ]:
print(f'Scale: {sample.pattern.scale}')
print(f'BG 0: {sample.backgrounds[0][0]}')
print(f'BG 1: {sample.backgrounds[0][1]}')

**Parameter object with varying accessors**

In [ ]:
print(f'Scale: {sample.pattern.scale}')
print(f'Scale: {sample.pattern.scale.value}')

The fit is quite good, but let's see if we can do better with a different optimizer.

## Change the optimizer to `bumps`

In [ ]:
print("available minimizers:", f.available_engines)
print()
print("current minimizer:", f.current_engine.name)
print("available methods of current minimizer:", f.available_methods())

In [ ]:
print("switch minimizer")
f.switch_engine('bumps')
f_method = 'lm'
print("current minimizer:", f.current_engine.name)
print("available methods of current minimizer:", f.available_methods())

**Rerun fitting** (takes a while!)

This seems completely broken and needs fixing. The fitting just hangs.

In [ ]:
result = f.fit(data_x, data_y, weights=1/data_e, method=f_method)

if result.success:
    print("The fit has been successful: {}".format(result.success))
    print("The gooodness of fit is: {}".format(result.goodness_of_fit))
    
sim_y_data = calculator.fit_func(data_x)

In [ ]:
%matplotlib notebook
plt.plot(data_x, data_y, label='Experimental')
plt.plot(data_x, sim_y_data, label='Best Fit')
plt.legend()

In [ ]:
print(f'Scale: {sample.pattern.scale}')
print(f'BG 0: {sample.backgrounds[0][0]}')
print(f'BG 1: {sample.backgrounds[0][1]}')
#print(f'Res U: {sample.parameters.resolution_u}')
#print(f'Res V: {sample.parameters.resolution_v}')
#print(f'Res W: {sample.parameters.resolution_w}')